In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/ruslansafarov/Documents/GitHub/BasicRepository/Unit_3/main_task.csv')

In [3]:
# посчитаем число пустых ячеек в каждой колонке
for col in df.columns:
    print(col, df[col].isna().sum())

Restaurant_id 0
City 0
Cuisine Style 9283
Ranking 0
Rating 0
Price Range 13886
Number of Reviews 2543
Reviews 0
URL_TA 0
ID_TA 0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [4]:
# узнаем типы значений в колонках
for col in df.columns:
    print(type(df[col].iloc[0]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'str'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'str'>


In [39]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
X = df.drop(['Restaurant_id', 
             'Rating', 
             'City',
             'Cuisine Style',
             'CuisineList',
             'Price Range',
             'Reviews',
             'DateTime1',
             'DateTime2',
             'URL_TA',
             'ID_TA'], axis = 1) 

In [40]:
X.head(2)

,Ranking,Number of Reviews,high_price,low_price,middle_price,unknown_price,Berlin,London,Madrid,Milan,...,CuisineCnt,UnknownCausine,c_European,c_Gluten Free Options,c_Italian,c_Mediterranean,c_Other,c_Vegan Options,c_Vegetarian Friendly,DaysBetween
0,5570.0,194.0,0,0,1,0,0,0,0,0,...,3,0,1,0,0,0,1,0,0,41.0
1,1537.0,10.0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,382.0


In [41]:
# заполняем все пропуски средними значениями
# понимая, что в даммис переменных прпусков нет
for col in X.columns:
    val = X[col].mean()
    X[col].fillna(val, inplace=True)

y = df['Rating']  
  
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
  
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  

In [42]:
X['Number of Reviews'].isna().sum()

0

In [43]:
X['Ranking'].isna().sum()

0

In [44]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
  
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
  
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test) 

In [45]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются  
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.  
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  

MAE: 0.2810001202380953


# НАЧИНАЕМ ПОДГОТОВКУ ДАННЫХ И ФИЧАИНЖИНИРИНГ

In [7]:
df['Price Range'].value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [8]:
len(df[df['Price Range'] == '$$ - $$$'])

18412

In [9]:
def renprice(cell):
    if cell=='$':
        return 'low_price'
    elif cell=='$$ - $$$':
        return 'middle_price'
    elif cell=='$$$$':
        return 'high_price'
    else:
        return 'unknown_price'

df['Price Range'] = df['Price Range'].apply(renprice)

In [10]:
df = df.join(pd.get_dummies(df['Price Range']))

In [11]:
df.City.unique()

array(['Paris', 'Stockholm', 'London', 'Berlin', 'Munich', 'Oporto',
       'Milan', 'Bratislava', 'Vienna', 'Rome', 'Barcelona', 'Madrid',
       'Dublin', 'Brussels', 'Zurich', 'Warsaw', 'Budapest', 'Copenhagen',
       'Amsterdam', 'Lyon', 'Hamburg', 'Lisbon', 'Prague', 'Oslo',
       'Helsinki', 'Edinburgh', 'Geneva', 'Ljubljana', 'Athens',
       'Luxembourg', 'Krakow'], dtype=object)

In [12]:
import numpy as np
cities_freq = list(df.City.value_counts())

In [13]:
# ‘linear’, ‘lower’, ‘higher’, ‘midpoint’, ‘nearest’
np.percentile(cities_freq, 90, interpolation='nearest')

2734

In [14]:
top_cities_count = int(np.percentile(cities_freq, 80))
top_cities_count

2078

In [15]:
all_cities = df.City.value_counts().index
vcounts_cities = df.City.value_counts() 
top_cities = vcounts_cities[vcounts_cities>=top_cities_count].index
#cities_to_trow_away = list(set(all_cities) - set(top_cities))
df.loc[~df['City'].isin(top_cities), 'City'] = "UnknownCity"

In [16]:
df = df.join(pd.get_dummies(df.City, drop_first=True))

In [17]:
def str_to_list(cell):
    bad_chars = "[/\.<>]'"
    lst = ['UnknownCausine']

    if not isinstance(cell, float):
        for ch in bad_chars:
            cell = cell.replace(ch, '')
        lst = cell.split(', ')
        
        #добавим префикс, чтобы не путать визуально с городами
        for i in range(len(lst)):
            newstr = "c_" + lst[i] 
            lst[i] = newstr 

    return lst

cuisine_list = df['Cuisine Style'].apply(str_to_list)
df['CuisineList'] = cuisine_list

In [18]:
lst = []
for itm in cuisine_list:
    if itm[0] !='UnknownCausine':
        lst.append(len(itm))
    else:
        lst.append(0)
df['CuisineCnt'] = lst

In [19]:
import collections
c = collections.Counter()
for lst in cuisine_list:
    for itm in lst:
        c[itm] += 1

In [20]:
c.most_common()

[('c_Vegetarian Friendly', 11189),
 ('c_European', 10060),
 ('UnknownCausine', 9283),
 ('c_Mediterranean', 6277),
 ('c_Italian', 5964),
 ('c_Vegan Options', 4486),
 ('c_Gluten Free Options', 4113),
 ('c_Bar', 3297),
 ('c_French', 3190),
 ('c_Asian', 3011),
 ('c_Pizza', 2849),
 ('c_Spanish', 2798),
 ('c_Pub', 2449),
 ('c_Cafe', 2325),
 ('c_Fast Food', 1705),
 ('c_British', 1595),
 ('c_International', 1584),
 ('c_Seafood', 1505),
 ('c_Japanese', 1464),
 ('c_Central European', 1393),
 ('c_American', 1315),
 ('c_Sushi', 1156),
 ('c_Chinese', 1145),
 ('c_Portuguese', 1107),
 ('c_Indian', 1041),
 ('c_Middle Eastern', 782),
 ('c_Thai', 743),
 ('c_Wine Bar', 697),
 ('c_German', 662),
 ('c_Healthy', 620),
 ('c_Greek', 604),
 ('c_Halal', 598),
 ('c_Czech', 595),
 ('c_Fusion', 577),
 ('c_Steakhouse', 573),
 ('c_Barbecue', 555),
 ('c_Contemporary', 523),
 ('c_Vietnamese', 513),
 ('c_Eastern European', 496),
 ('c_Soups', 494),
 ('c_Grill', 490),
 ('c_Gastropub', 471),
 ('c_Mexican', 445),
 ('c_Turk

In [21]:
set_of_causines = set(c)
val_of_causines = list(c.values())

In [22]:
# создадим дамми переменные для кухонь мира встречающихся в 95% случаях выборки
# ‘linear’, ‘lower’, ‘higher’, ‘midpoint’, ‘nearest’
perc = np.percentile(val_of_causines, 95, interpolation='nearest')

most_popular_causine = []
for itm in c:
    if c[itm]>=perc:
        most_popular_causine.append(itm)

df['CuisineList'] = df['CuisineList'].map(lambda x: list(set([i if i in most_popular_causine else 'c_Other' for i in x])))
        


In [23]:
df = df.join(pd.get_dummies(df['CuisineList'].apply(pd.Series).stack()).sum(level=0))

In [24]:
df.head(8)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,...,CuisineList,CuisineCnt,UnknownCausine,c_European,c_Gluten Free Options,c_Italian,c_Mediterranean,c_Other,c_Vegan Options,c_Vegetarian Friendly
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,middle_price,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,...,"[c_European, c_Other]",3,0,1,0,0,0,1,0,0
1,id_1535,UnknownCity,NaN,1537.0,4.0,unknown_price,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,...,[UnknownCausine],0,1,0,0,0,0,0,0,0
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,high_price,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,...,"[c_Other, c_Gluten Free Options, c_Vegetarian ...",7,0,0,1,0,0,1,1,1
3,id_3456,Berlin,NaN,3458.0,5.0,unknown_price,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,...,[UnknownCausine],0,1,0,0,0,0,0,0,0
4,id_615,UnknownCity,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,middle_price,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,...,"[c_Other, c_Vegetarian Friendly]",3,0,0,0,0,0,1,0,1
5,id_1418,UnknownCity,NaN,1419.0,3.0,unknown_price,2.0,"[['There are better 3 star hotel bars', 'Amazi...",/Restaurant_Review-g189180-d12503536-Reviews-D...,d12503536,...,[UnknownCausine],0,1,0,0,0,0,0,0,0
6,id_1720,Milan,"['Italian', 'Pizza']",1722.0,4.0,low_price,50.0,"[['Excellent simple local eatery.', 'Excellent...",/Restaurant_Review-g187849-d5808504-Reviews-Pi...,d5808504,...,"[c_Other, c_Italian]",2,0,0,0,1,0,1,0,0
7,id_825,UnknownCity,['Italian'],826.0,3.0,unknown_price,9.0,"[['Wasting of money', 'excellent cuisine'], ['...",/Restaurant_Review-g274924-d3199765-Reviews-Ri...,d3199765,...,[c_Italian],1,0,0,0,1,0,0,0,0


In [25]:
df['CuisineCnt'].sum()/40000

2.390325

In [26]:
import re
reg = re.compile(r'\d{2}/\d{2}/\d{4}')

reviews = df['Reviews'].dropna().apply(lambda x: ['default'] if pd.isnull(x) else reg.findall(x))

In [27]:
len(reviews)

40000

In [28]:
df['DaysBetween'] = abs(pd.to_datetime(reviews.str[1]) - pd.to_datetime(reviews.str[0])).dropna()

In [29]:
df['DaysBetween'] = df['DaysBetween'].apply(lambda x: x.days)

In [30]:
df['DaysBetween']

0          41.0
1         382.0
2           2.0
3           NaN
4         272.0
          ...  
39995      34.0
39996       9.0
39997    3127.0
39998      23.0
39999    1306.0
Name: DaysBetween, Length: 40000, dtype: float64

In [31]:
from datetime import datetime
df['DateTime1'] = pd.to_datetime(reviews.str[0])
df['DateTime2'] = pd.to_datetime(reviews.str[1])
# готовим данные, чтобы произвести вычисления
df['DateTime1'].fillna(-1, inplace=True)
df['DateTime2'].fillna(-1, inplace=True)


df['Pastdays'] = df['DateTime1'] if (df['DateTime1']-df['DateTime1']).dt.days>0 else df['DateTime2']

AttributeError: Can only use .dt accessor with datetimelike values

In [32]:
reviews.sample(15)

5465     [11/19/2017, 11/11/2017]
33002    [01/09/2018, 08/17/2017]
14583    [03/09/2017, 10/14/2016]
7492                           []
16222    [02/18/2018, 11/20/2017]
18035    [05/14/2017, 04/24/2017]
36264    [09/06/2017, 08/09/2017]
19612                          []
19242    [09/20/2017, 05/26/2017]
10866                          []
444      [07/04/2017, 09/05/2016]
7729     [01/01/2018, 11/27/2017]
12858    [09/30/2017, 01/11/2017]
30878    [11/10/2015, 03/28/2015]
39765    [12/22/2017, 10/11/2017]
Name: Reviews, dtype: object

In [33]:
c=pd.to_datetime(reviews.str[0]).dropna()
d=pd.to_datetime(reviews.str[1]).dropna()

In [34]:
len(c)

33529

In [35]:
d.max()

Timestamp('2018-02-26 00:00:00')

In [36]:
c.max()

Timestamp('2018-02-26 00:00:00')

In [37]:
f=c-d
#f.sort_values(ascending=False)
f.max()

Timedelta('3207 days 00:00:00')

In [38]:
reviews

0        [12/31/2017, 11/20/2017]
1        [07/06/2017, 06/19/2016]
2        [01/08/2018, 01/06/2018]
3                              []
4        [11/18/2017, 02/19/2017]
                   ...           
39995    [12/16/2017, 11/12/2017]
39996    [12/21/2017, 12/12/2017]
39997    [11/03/2016, 04/12/2008]
39998    [07/11/2017, 06/18/2017]
39999    [01/31/2016, 07/04/2012]
Name: Reviews, Length: 40000, dtype: object